# Import packages

In [42]:
# Set auto reload for python functions
%reload_ext autoreload
%autoreload 2

# Import packages
import os
import glob
import tqdm
import tensorflow as tf
from collections import defaultdict
from time import time
from I_matrix import get_act_sizes
from I_matrix import get_weight_sizes
from I_matrix import get_channel_sizes
from I_matrix import gen_I_matrix
from data_parser import _parse_function
from keras.applications.inception_v3 import preprocess_input

# GoogLeNet # Need to install lucid with the command: `pip install lucid`
import lucid.modelzoo.vision_models as models
import lucid.optvis.render as render

# Setup

In [45]:
# GPU to use
use_gpu = False
gpu = 0

# Input directory path
input_dir_path = '/Users/haekyu/data/imagenet-tf-records'

# Output directory path
output_dir_path = '/Users/haekyu/data/massif/I-mat'

# Layer info
layer_sizes = {
    'mixed3a': 256,
    'mixed3b': 480,
    'mixed4a': 508,
    'mixed4b': 512,
    'mixed4c': 512,
    'mixed4d': 528,
    'mixed4e': 832,
    'mixed5a': 832,
    'mixed5b': 1024
}
mixed_layers = list(layer_sizes.keys())

# Class info
num_class = 1000

# Pick a layer
layer = 'mixed3b'

# The number of top neurons to select
k = 5

# Batch size
batch = 500

In [5]:
# Assign a GPU
if use_gpu:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)

In [6]:
# Outlier nodes
outlier_nodes = ['mixed3a-67', 'mixed3a-190', 'mixed3b-390', 'mixed3b-399', 'mixed3b-412']

## Import GoogLeNet

In [8]:
# Import googlenet
googlenet = models.InceptionV1()
googlenet.load_graphdef()
nodes = googlenet.graph_def.node

In [14]:
# Googlenet size info
layer_fragment_sizes = {layer: get_channel_sizes(layer, nodes) for layer in mixed_layers}
weight_sizes = get_weight_sizes(nodes, mixed_layers)
act_sizes = get_act_sizes(weight_sizes, mixed_layers)

# Get I-matrix

In [48]:
# Set auto reload for python functions
%reload_ext autoreload
%autoreload 2
from I_matrix import gen_I_matrix
I_layer = gen_I_matrix(layer, k, googlenet, batch, input_dir_path, mixed_layers, layer_sizes, act_sizes, layer_fragment_sizes, outlier_nodes, num_class)